# Create data flow and data jobs

In [11]:
import uuid
from datetime import datetime, timezone
from datahub.api.entities.datajob import DataFlow, DataJob
from datahub.api.entities.dataprocess.dataprocess_instance import (
    DataProcessInstance,
    InstanceRunResult,
)
from datahub.emitter.rest_emitter import DatahubRestEmitter

In [12]:
token =

In [13]:
emitter = DatahubRestEmitter(gms_server="http://localhost:8080", extra_headers={'Authorization':  'Bearer '+token,
                                            'Content-Type': 'application/json',})

In [14]:
jobFlow = DataFlow(env="EI", orchestrator="Azure Data Factory", id="Centers Analysis Flow")
jobFlow.emit(emitter)

In [15]:
dataJob = DataJob(flow_urn=jobFlow.urn, id="job_dim_center", name="Populate Centers Dimension")
dataJob.properties["Dimension Type"] = "Delta"
dataJob.emit(emitter)

In [16]:
dataJob2 = DataJob(flow_urn=jobFlow.urn, id="job_dim_employee", name="Populate Employee Dimension")
dataJob2.properties["Dimension Type"] = "Delta"
dataJob2.upstream_urns.append(dataJob.urn)
dataJob2.emit(emitter)

In [17]:
dataJob3 = DataJob(flow_urn=jobFlow.urn, id="job_dim_service", name="Populate Service Dimension")
dataJob3.properties["Dimension Type"] = "Slowly Changing Dimension Type 1"
dataJob3.upstream_urns.append(dataJob2.urn)
dataJob3.emit(emitter)

In [18]:
dataJob4 = DataJob(flow_urn=jobFlow.urn, id="job_dim_trs_type", name="Populate Transaction Type Dimension")
dataJob4.properties["Dimension Type"] = "Delta"
dataJob4.upstream_urns.append(dataJob3.urn)
dataJob4.emit(emitter)

In [19]:
dataJob5 = DataJob(flow_urn=jobFlow.urn, id="job_dim_file_type", name="Populate File Type Dimension")
dataJob5.properties["Dimension Type"] = "Delta"
dataJob5.upstream_urns.append(dataJob4.urn)
dataJob5.emit(emitter)

In [20]:
dataJob6 = DataJob(flow_urn=jobFlow.urn, id="job_fact_centers", name="Populate Centers Fact")
dataJob6.properties["Fact Type"] = "Transactional"
dataJob6.upstream_urns.append(dataJob5.urn)
dataJob6.emit(emitter)